In [1]:
# orb brute force works
import numpy as np
import cv2

MIN_MATCH_COUNT = 10

cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
orb = cv2.ORB_create(nfeatures=np.int(1e4))
#orb = cv2.ORB_create()
ret, frame = cap.read()
scale = 1
frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
print(frame.shape) # (720, 1280, 3)
half = 50 // scale
y, x = 280 // scale, 600 // scale
crop = frame[y - half:y + half, x - half:x + half]
kp_prev, des_prev = orb.detectAndCompute(crop,None)
crop = cv2.drawKeypoints(crop, kp_prev, None, color=(0,255,0), flags=0)
cv2.imshow('crop', crop)
cv2.waitKey(0)
cv2.destroyAllWindows()

while True:
    ret, frame = cap.read()
    
    frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
    if ret == True:
        kp_frame, des_frame = orb.detectAndCompute(frame, None)
        
        
        # create BFMatcher object
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        # Match descriptors.
        matches = bf.match(des_prev,des_frame)
        # Sort them in the order of their distance.
        matches = sorted(matches, key = lambda x:x.distance)
        # Draw first 10 matches.
        frame = cv2.drawKeypoints(frame, kp_frame, None, color=(0,255,0), flags=0)
        out = cv2.drawMatches(crop,kp_prev,frame,kp_frame,matches[:10], None, flags=2)
        cv2.imshow('out', out)
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()

(720, 1280, 3)


In [61]:
'''
import numpy as np
import cv2

MIN_MATCH_COUNT = 10

cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
#detector = cv2.ORB_create(nfeatures=np.int(1e4))
#detector = cv2.ORB_create()
detector = cv2.xfeatures2d.SIFT_create()
for i in range(100):
    ret, frame = cap.read()
scale = 3
frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
print(frame.shape) # (720, 1280, 3)
half = 50 // scale
y, x = 340 // scale, 680 // scale
crop = frame[y - half:y + half, x - half:x + half]
kp_prev, des_prev = detector.detectAndCompute(crop,None)
crop = cv2.drawKeypoints(crop, kp_prev, None, color=(0,255,0), flags=0)
cv2.imshow('crop', crop)
cv2.waitKey(0)
cv2.destroyAllWindows()
while True:
    ret, frame = cap.read()
    
    frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
    if ret == True:
        kp_frame, des_frame = detector.detectAndCompute(frame, None)
        
        
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des_prev,des_frame,k=2)
        
        # Need to draw only good matches, so create a mask
        matchesMask = [[0,0] for i in range(len(matches))]
        # ratio test as per Lowe's paper
        good = []
        for i,(m,n) in enumerate(matches):
            if m.distance < 0.7*n.distance:
                matchesMask[i]=[1,0]
                good.append((m, n))
        draw_params = dict(matchColor = (0,255,0),
                           singlePointColor = (255,0,0),
                           matchesMask = matchesMask,
                           flags = 0)
        out = cv2.drawMatchesKnn(crop,kp_prev,frame,kp_frame,matches,None,**draw_params)
        cv2.imshow('out', out)
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()
'''

(240, 427, 3)


In [19]:
# camshift add
import numpy as np
import cv2

cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
orb = cv2.ORB_create(nfeatures=np.int(1e4))
#orb = cv2.ORB_create()
ret, frame = cap.read()
scale = 1
frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale)
print(frame.shape) # (720, 1280, 3)
half = 50 // scale
y, x = 280 // scale, 600 // scale
crop = frame[y - half:y + half, x - half:x + half]
kp_prev, des_prev = orb.detectAndCompute(crop,None)
crop = cv2.drawKeypoints(crop, kp_prev, None, color=(0,255,0), flags=0)
















r,h,c,w = y - half, 2 * half,x - half,2 * half  # simply hardcoded the values
track_window = (c,r,w,h)
# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]
hsv_roi = roi[:, :, 0, True]
hsv_roi[True] = 255
roi_hist = cv2.calcHist([hsv_roi],[0],None, [180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )



'''
cv2.imshow('bitwise_not', bitwise_not)
cv2.imshow('roi_hist', roi_hist)
cv2.imshow('hsv_roi', hsv_roi)
'''
cv2.imshow('crop', crop)
cv2.waitKey(0)
cv2.destroyAllWindows()

while True:
    ret, frame = cap.read()
    
    frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale)
    if ret == True:
        kp_frame, des_frame = orb.detectAndCompute(frame, None)
        
        
        # create BFMatcher object
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        # Match descriptors.
        matches = bf.match(des_prev,des_frame)
        # Sort them in the order of their distance.
        matches = sorted(matches, key = lambda x:x.distance)
        # Draw first 10 matches.
        out = cv2.drawMatches(crop,kp_prev,frame,kp_frame,matches[:10], None, flags=2)
        
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        
        
        
        cv2.imshow('out', out)
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()

(720, 1280, 3)


In [129]:
test1 = test = frame.copy()
test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2GRAY)
test1[True] = 255
roi_hist = cv2.calcHist([test1],[0],None,[180],[0,180])
cv2.imshow('test1', test1)
print(roi_hist)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

In [3]:
# camshift
import numpy as np
import cv2
cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
# take first frame of the video
ret,frame = cap.read()
# setup initial location of window
scale = 1
half = 50 // scale
y, x = 280 // scale, 600 // scale
r,h,c,w = y - half, 2 * half,x - half,2 * half  # simply hardcoded the values
track_window = (c,r,w,h)
# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret ,frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv2.imshow('img2',img2)
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()

In [28]:
# camshift add
import numpy as np
import cv2

cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
orb = cv2.ORB_create(nfeatures=np.int(1e5))
#orb = cv2.ORB_create()
ret, frame = cap.read()
scale = 1
frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale)
print(frame.shape) # (720, 1280, 3)
half = 50 // scale
y, x = 280 // scale, 600 // scale
crop = frame[y - half:y + half, x - half:x + half]
kp_prev, des_prev = orb.detectAndCompute(crop,None)
crop = cv2.drawKeypoints(crop, kp_prev, None, color=(0,255,0), flags=0)

ret, frame = cap.read()    
frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale)
half2 = 50 // scale
crop2 = frame[y - half2:y + half2, x - half2:x + half2]
kp_frame, des_frame = orb.detectAndCompute(crop2, None)
cv2.imshow('crop', crop)
                                           
# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(des_prev,des_frame)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first 10 matches.
out = cv2.drawMatches(crop,kp_prev,crop2,kp_frame,matches[:100], None, flags=2)
cv2.imshow('out', out)
cv2.waitKey(0)
cv2.destroyAllWindows()

(720, 1280, 3)


In [32]:
help(matches[0])

Help on DMatch object:

class DMatch(builtins.object)
 |  Methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  distance
 |      distance
 |  
 |  imgIdx
 |      imgIdx
 |  
 |  queryIdx
 |      queryIdx
 |  
 |  trainIdx
 |      trainIdx



In [35]:
for m in matches[:10]:
    print(kp_frame[m.trainIdx].pt)

(48.0, 40.80000305175781)
(40.80000305175781, 49.20000076293945)
(41.0, 50.0)
(39.60000228881836, 51.60000228881836)
(37.0, 55.0)
(34.0, 47.0)
(38.0, 52.0)
(66.0, 65.0)
(40.80000305175781, 58.80000305175781)
(34.0, 37.0)
